<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/ACL-cleanup/Copy_of_WEAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEAT Algorithm
## Test Statistic

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# returns s(w, A, B) for all w in W (passed as argument). Shape: n_words (in W) x 1
def swAB(W, A, B):
  #Calculate cosine-similarity between W and A, W and B
  #print("W: ", W.shape, " A: ", A.shape, " B: ", B.shape)
  WA = cosine_similarity(W,A)
  WB = cosine_similarity(W,B)
  #print('WA shape: ', WA.shape)
  #Take mean along columns
  WAmean = np.mean(WA, axis = 1)
  WBmean = np.mean(WB, axis = 1)
  
  #print('sWAB shape: ', WAmean.shape)
  
  return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
  return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

def weat_effect_size(X, Y, A, B, embd):
  #Convert the set of words to matrix
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  # Find X U Y
  XuY = list(set(X).union(Y))
  XuYmat = []
  for w in XuY:
    if w.lower() in embd:
      XuYmat.append(embd[w.lower()])
  XuYmat = np.array(XuYmat)
#   print('X U Y Shape: ', XuYmat.shape)
#   print('X Shape: ', Xmat.shape)
#   print('Y Shape: ', Ymat.shape)
#   print('A Shape: ', Amat.shape)
#   print('B Shape: ', Bmat.shape)
  
  d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
  
  return d

## P-Value

In [0]:
import random
from itertools import combinations, filterfalse

def random_permutation(iterable, r=None):
  pool = tuple(iterable)
  r = len(pool) if r is None else r
  return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample):
  size_of_permutation = min(len(X), len(Y))
  X_Y = X + Y
  test_stats_over_permutation = []
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  if not sample:
      permutations = combinations(X_Y, size_of_permutation)
  else:
      permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
      
  for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
    Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
    
  unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
  
  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  #print("All: ", test_stats_over_permutation)
  #print("Unpertrubed: ", unperturbed)
  return is_over.sum() / is_over.size

## Test on sample input

In [0]:
X = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"] #Instruments
Y = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"] #Weapons
A = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"] #Pleasant
B = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"] #Unpleasant

#Load word embeddings
#load gensim formatted Full Glove embeddings
#!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''


glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

print('WEAT d = ', weat_effect_size(X, Y, A, B, glove))
print('WEAT p = ', weat_p_value(X, Y, A, B, glove, 1000))

The glove embedding has been loaded!
WEAT d =  1.5495627
Unpertrubed:  2.2905553244054317
WEAT p =  0.0


## WEAT with conceptor debiased embeddings

### Compute the conceptor matrix for all words and gender specific words.

In [0]:
#Compute the conceptor matrix
def process_cn_matrix(subspace, alpha = 2):
  #print("starting...")
  #x = orig_embd.vectors
  #print(subspace.shape)
  
  #Calculate the correlation matrix
  R = subspace.dot(subspace.T)/(subspace.shape[1])
  #print("R calculated")
  
  #Calculate the conceptor matrix
  C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(subspace.shape[0])))
  #print("C calculated")
  
  #Calculate the negation of the conceptor matrix
  negC = np.eye(subspace.shape[0]) - C
  #print("negC calculated")
  
  return negC

def apply_conceptor(x, C):
  #Post-process the vocab matrix
  newX = (C @ x).T
  print(newX.shape)
  return newX

Load embeddings of all words from the ref. wordlist from a specific embedding

In [0]:
#Arguments - embd: The word embeddings in the form of a dict || wikiWordsPath: List of words to be considered
def load_all_vectors(embd, wikiWordsPath):
  all_words_index = {}
  all_words_mat = []
  with open(wikiWordsPath, "r+") as f_in:
    ind = 0
    for line in f_in:
      word = line.split(' ')[0]
      if word in embd:
        all_words_index[word] = ind
        all_words_mat.append(embd[word])
        ind = ind+1
        
  return all_words_index, all_words_mat

def load_subspace_vectors(embd, subspace_words):
  subspace_embd_mat = []
  ind = 0
  for word in subspace_words:
    if word in embd:
      subspace_embd_mat.append(embd[word])
      ind = ind+1
      
  return subspace_embd_mat

### Load all word lists - Subspace

In [0]:
# General word list
!wget https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20150602-words-frequency.txt
!git clone https://github.com/PrincetonML/SIF
    
# Gender word lists
!git clone https://github.com/uclanlp/gn_glove
!git clone https://github.com/uclanlp/corefBias
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
    
# our code for debiasing -- also includes word lists    
!git clone https://github.com/jsedoc/ConceptorDebias

In [0]:
from load_word_lists import *

gender_list_pronouns = WEATLists.W_7_Male_terms + WEATLists.W_7_Female_terms + WEATLists.W_8_Male_terms + WEATLists.W_8_Female_terms
gender_list_pronouns = list(set(gender_list_pronouns))

gender_list_extended = male_vino_extra + female_vino_extra + male_gnGlove + female_gnGlove
gender_list_extended = list(set(gender_list_extended))

gender_list_propernouns = male_cmu + female_cmu
gender_list_propernouns = list(set(gender_list_propernouns))

gender_list_all = gender_list_pronouns + gender_list_extended + gender_list_propernouns
gender_list_all = list(set(gender_list_all))

race_list = WEATLists.W_3_Unused_full_list_European_American_names + WEATLists.W_3_European_American_names + WEATLists.W_3_Unused_full_list_African_American_names + WEATLists.W_3_African_American_names + WEATLists.W_4_Unused_full_list_European_American_names + WEATLists.W_4_European_American_names + WEATLists.W_4_Unused_full_list_African_American_names + WEATLists.W_4_African_American_names + WEATLists.W_5_Unused_full_list_European_American_names + WEATLists.W_5_European_American_names + WEATLists.W_5_Unused_full_list_African_American_names + WEATLists.W_5_African_American_names 
race_list = list(set(race_list))

### Load different embeddings

Glove

In [0]:
#Load word embeddings
#download gensim formatted Full Glove embeddings
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

In [0]:
X = WEATLists.W_5_Unused_full_list_European_American_names
print(X)
a = [glove[w] for w in X if w.lower() in glove]
print(np.array(a).shape)
glove['Brad']
#glove['brad']

Word2vec

In [0]:
#load gensim formatted Full Word2vec embeddings
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
!gunzip GoogleNews-vectors-negative300.bin.gz
  
import gensim

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

word2vec = KeyedVectors.load_word2vec_format(resourceFile + 'GoogleNews-vectors-negative300.bin', binary=True)
print('The word2vec embedding has been loaded!')

Fasttext

In [0]:
!gdown https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh

import gensim
from gensim.models.keyedvectors import KeyedVectors  

resourceFile = ''
fasttext = KeyedVectors.load_word2vec_format(resourceFile + 'fasttext.bin', binary=True)
print('The fasttext embedding has been loaded!')

Elmo

In [0]:
!gdown https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU
  
import pickle
data = pickle.load(open("elmo_embeddings_emma_brown.pkl", "rb"))

def pick_embeddings(corpus, sent_embs):
    X = []
    labels = {}
    sents = []
    ind = 0
    for i, s in enumerate(corpus):
        for j, w in enumerate(s):
            X.append(sent_embs[i][j])
            if w.lower() in labels:
              labels[w.lower()].append(ind)
            else:
              labels[w.lower()] = [ind]
            sents.append(s)
            ind = ind + 1
    return (X, labels, sents)
  
def get_word_list(path):
    word_list = []
    with open(path, "r+") as f_in:
      for line in f_in:
        word = line.split(' ')[0]
        word_list.append(word.lower())

    return word_list

def load_subspace_vectors_contextual(all_mat, all_index, subspace_list):
    subspace_mat = []
    for w in subspace_list:
      if w.lower() in all_index:
        for i in all_index[w.lower()]:
          #print(type(i))
          subspace_mat.append(all_mat[i])
    #subspace_mat = [all_mat[i,:] for i in all_index[w.lower()] for w in subspace_list if w.lower() in all_index]
    print("Subspace: ", np.array(subspace_mat).shape)
    return subspace_mat

In [0]:
import nltk
from nltk.corpus import brown

#nltk.download('brown')

brown_corpus = brown.sents()
elmo = data['brown_embs']


In [0]:
print(np.array(elmo_brown_mat).shape)
print(len(list(elmo_brown_index.keys())))
print(len(wiki_words))

###Post-process (CN) all embeddings using a particular subspace

Test WEAT for a given pair of attribute and target pairs.

Note: To change the attribute and target word sets, change the values of X, Y, A, B. (It should be a list of words)

Subspace(s) to be tested upon can be changed by setting 'all_subspace' to a list of variables representing the subspace. Each of these variables should be a list of words for that subspace.

In [0]:
import numpy as np
from ConceptorDebias.conceptor_fxns import AND
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

#List of all embeddings to test on
all_embd = ['glove', 'word2vec', 'fasttext','elmo']

#List of all subspaces for gender
all_subspace = ['without_conceptor','gender_list_pronouns', 'gender_list_extended','gender_list_propernouns', 'gender_list_all', 'gender_list_and']

#List of all subsoaces for race
# all_subspace = ['without_conceptor', 'race_list']

# X = WEATLists.W_6_Career
# Y = WEATLists.W_6_Family
# A = WEATLists.W_6_Male_names
# B = WEATLists.W_6_Female_names

X = WEATLists.W_5_Unused_full_list_European_American_names
Y = WEATLists.W_5_Unused_full_list_African_American_names
A = WEATLists.W_5_Pleasant
B = WEATLists.W_5_Unpleasant

# X = WEATLists.W_8_Science
# Y = WEATLists.W_8_Arts
# A = WEATLists.W_8_Male_terms
# B = WEATLists.W_8_Female_terms


results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    #wiki_words = get_word_list('SIF/auxiliary_data/enwiki_vocab_min200.txt')
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_conceptor' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_conceptor':
      #CN all word embeddings using the respective subspace
      if subspace == 'gender_list_and':
        if embd == 'elmo' or embd == 'bert':
          subspace_words_mat1 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_pronouns)
          cn1 = process_cn_matrix(np.array(subspace_words_mat1).T, alpha = 8)

          subspace_words_mat2 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_extended)
          cn2 = process_cn_matrix(np.array(subspace_words_mat2).T, alpha = 3)

          subspace_words_mat3 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_propernouns)
          cn3 = process_cn_matrix(np.array(subspace_words_mat3).T, alpha = 10)

          cn = AND(cn1, AND(cn2, cn3))
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
          print("All mat CN: ", np.array(all_words_cn).shape)
        else:
          subspace_words_mat1 = load_subspace_vectors(curr_embd, gender_list_pronouns)
          cn1 = process_cn_matrix(np.array(subspace_words_mat1).T)

          subspace_words_mat2 = load_subspace_vectors(curr_embd, gender_list_extended)
          cn2 = process_cn_matrix(np.array(subspace_words_mat2).T)

          subspace_words_mat3 = load_subspace_vectors(curr_embd, gender_list_propernouns)
          cn3 = process_cn_matrix(np.array(subspace_words_mat3).T)

          cn = AND(cn1, AND(cn2, cn3))
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      else:
        #Load all embeddings of the subspace as a matrix
        if embd == 'elmo' or embd == 'bert':
          subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
          cn = process_cn_matrix(np.array(subspace_words_mat).T, alpha = 6)
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
          print("Subspace mat: ", np.array(subspace_words_mat).shape)
        else:
          subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
          cn = process_cn_matrix(np.array(subspace_words_mat).T)
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
#     print("LAST: ", np.array(all_words["a"]).shape)
    if subspace == 'without_conceptor':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(X, Y, A, B, all_words)
      p_cn = weat_p_value(X, Y, A, B, all_words, 1000)
      print("Without conceptor")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(X, Y, A, B, all_words)
      p_cn = weat_p_value(X, Y, A, B, all_words, 1000)
      print("With conceptor: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

## WEAT algorithm from GITHUB gist
Ref: https://gist.github.com/SandyRogers/e5c2e938502a75dcae25216e4fae2da5

In [0]:
class WEATTest(object):
    """
    Perform WEAT (Word Embedding Association Test) bias tests on a language model.
    Follows from Caliskan et al 2017 (10.1126/science.aal4230).
    """
    
    instruments = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"]
    weapons = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"]
    flowers = ["aster", "clover", "hyacinth", "marigold", "poppy", "azalea", "crocus", "iris", "orchid", "rose", "blue-bell", "daffodil", "lilac", "pansy", "tulip", "buttercup", "daisy", "lily", "peony", "violet", "carnation", "gladiola", "magnolia", "petunia", "zinnia"]
    insects = ["ant", "caterpillar", "flea", "locust", "spider", "bedbug", "centipede", "fly", "maggot", "tarantula", "bee", "cockroach", "gnat", "mosquito", "termite", "beetle", "cricket", "hornet", "moth", "wasp", "blackfly",
    "dragonfly", "horsefly", "roach", "weevil"]
    pleasant = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"]
    unpleasant = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"]
    math = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
    arts = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
    male = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
    female = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
    science = ["science", "technology", "physics", "chemistry", "Einstein", "NASA", "experiment", "astronomy"]
    arts_2 = ["poetry", "art", "Shakespeare", "dance", "literature", "novel", "symphony", "drama"]
    male_2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
    female_2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "he"]
    mental_disease = ["sad", "hopeless", "gloomy", "tearful", "miserable", "depressed"]
    physical_disease = ["sick", "illness", "influenza", "disease", "virus", "cancer"]
    temporary = ["impermanent", "unstable", "variable", "fleeting", "short-term", "brief", "occasional"]
    permanent = ["stable", "always", "constant", "persistent", "chronic", "prolonged", "forever"]
    
    def __init__(self, model):
        """Setup a Word Embedding Association Test for a given spaCy language model.
        
        EXAMPLE:
            >>> nlp = spacy.load('en_core_web_md')
            >>> test = WEATTest(nlp)
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
        """
        self.model = model

    @staticmethod
    def word_association_with_attribute(self, w, A, B):
        return np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(a).reshape(1,-1)) for a in A]) - np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(b).reshape(1,-1)) for b in B])

    @staticmethod
    def differential_assoication(self, X, Y, A, B):
        return np.sum([self.word_association_with_attribute(self, x, A, B) for x in X]) - np.sum([self.word_association_with_attribute(self, y, A, B) for y in Y])

    @staticmethod
    def weat_effect_size(self, X, Y, A, B):
        return (
            np.mean([self.word_association_with_attribute(self, x, A, B) for x in X]) -
            np.mean([self.word_association_with_attribute(self, y, A, B) for y in Y])
        ) / np.std([self.word_association_with_attribute(self, w, A, B) for w in X + Y])

    @staticmethod
    def random_permutation(self, iterable, r=None):
        pool = tuple(iterable)
        r = len(pool) if r is None else r
        return tuple(random.sample(pool, r))

    @staticmethod
    def weat_p_value(self, X, Y, A, B, sample):
        size_of_permutation = min(len(X), len(Y))
        X_Y = X + Y
        observed_test_stats_over_permutations = []

        if not sample:
            permutations = combinations(X_Y, size_of_permutation)
        else:
            permutations = [self.random_permutation(self, X_Y, size_of_permutation) for s in range(sample)]
        print(np.array(X_Y).shape)
        for Xi in permutations:
            Yi = filterfalse(lambda w:w in Xi, X_Y)
            observed_test_stats_over_permutations.append(self.differential_assoication(self, Xi, Yi, A, B))

        unperturbed = self.differential_assoication(self, X, Y, A, B)
        is_over = np.array([o > unperturbed for o in observed_test_stats_over_permutations])
        return is_over.sum() / is_over.size

    @staticmethod
    def weat_stats(X, Y, A, B, self, sample_p=None):
        test_statistic = self.differential_assoication(self, X, Y, A, B)
        effect_size = self.weat_effect_size(self, X, Y, A, B)
        p = self.weat_p_value(self, X, Y, A, B, sample=sample_p)
        return test_statistic, effect_size, p

    def run_test(self, target_1, target_2, attributes_1, attributes_2, sample_p=None):
        """Run the WEAT test for differential association between two 
        sets of target words and two seats of attributes.
        
        EXAMPLE:
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
            >>> test.run_test(a, b, c, d, sample_p=1000) # use 1000 permutations for p-value calculation
            >>> test.run_test(a, b, c, d, sample_p=None) # use all possible permutations for p-value calculation
            
        RETURNS:
            (d, e, p). A tuple of floats, where d is the WEAT Test statistic, 
            e is the effect size, and p is the one-sided p-value measuring the
            (un)likeliness of the null hypothesis (which is that there is no
            difference in association between the two target word sets and
            the attributes).
            
            If e is large and p small, then differences in the model between 
            the attribute word sets match differences between the targets.
        """
        X = [list(self.model[w]) for w in target_1]
        Y = [list(self.model[w]) for w in target_2]
        A = [list(self.model[w]) for w in attributes_1]
        B = [list(self.model[w]) for w in attributes_2]
        print(X)
        return self.weat_stats(X, Y, A, B, self, sample_p)


## WEAT on BERT embeddings

In [0]:
import os

#Load all bert embeddings in a dictonary
all = {}
for filename in os.listdir('/home/saketk/bert'):
  all[filename] = pickle.load(open(filename, "rb"))

#Load all embeddings in a matrix and a dictonary of words to row numbers
res = all['big_bert_gender_list_extended.pkl']['type_embedings']
w = []
for name in all:
  res = np.concatenate((res, all[name]['type_embedings']))
  w += all[name]['words']
print(res.shape)
print(len(set(w)))
w = [aaa.lower() for aaa in w]

#Load all BERT conceptors
cn_pronouns = all['big_bert_gender_list_pronouns.pkl']['GnegC']
cn_propernouns = all['big_bert_gender_list_propernouns.pkl']['GnegC']
cn_extended = all['big_bert_gender_list_extended.pkl']['GnegC']
cn_all = all['big_bert_gender_list_all.pkl']['GnegC']
cn_race = all['big_bert_race_list.pkl']['GnegC']

print(np.array(cn_all).shape)

Test Debiasing on BERT Embeddings using WEAT

In [0]:
import numpy as np
from ConceptorDebias.conceptor_fxns import AND
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['bert']
all_subspace = ['without_conceptor','gender_list_pronouns', 'gender_list_extended','gender_list_propernouns', 'gender_list_all', 'gender_list_and']
# all_subspace = ['without_conceptor', 'race_list']

all_words_mat = res
all_words_index = {}
print(len(set(w)))
for i,a in enumerate(w):
  all_words_index[a] = i
# all_words_index = {wr:i for i,wr in enumerate(w)}
print("All_words_mat: ", all_words_mat.shape)
print("Index: ", len(all_words_index.keys()))
career = WEATLists.W_8_Science
family = WEATLists.W_8_Arts
male = WEATLists.W_8_Male_terms
female = WEATLists.W_8_Female_terms

# white = WEATLists.W_5_Unused_full_list_European_American_names
# black = WEATLists.W_5_Unused_full_list_African_American_names
# pleasant = WEATLists.W_5_Pleasant
# unpleasant = WEATLists.W_5_Unpleasant
embd = 'bert'
results = []
for subspace in all_subspace:
    
  if subspace != 'without_conceptor' and subspace != 'gender_list_and':
    subspace_words_list = eval(subspace)

  if subspace != 'without_conceptor':
    #CN all word embeddings using the respective subspace
    if subspace == 'gender_list_and':
      cn = AND(cn_pronouns, AND(cn_extended, cn_propernouns))
      all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      print("All mat CN: ", np.array(all_words_cn).shape)
    else:
      #Load all embeddings of the subspace as a matrix
      if subspace == 'gender_list_pronouns':
        cn = cn_pronouns
      elif subspace == 'gender_list_propernouns':
        cn = cn_propernouns
      elif subspace == 'gender_list_extended':
        cn = cn_extended
      elif subspace == 'gender_list_all':
        cn = cn_all
        
      print("CN shape: ", np.array(cn).shape)
      all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      print("Subspace mat: ", np.array(subspace_words_mat).shape)
      
  else:
    all_words_cn = all_words_mat
  all_words_cn = np.array(all_words_cn)
  print("All CN: ", all_words_cn.shape)
  #Store all conceptored words in a dictonary
  all_words = {}
  for word, index in all_words_index.items():
    all_words[word] = all_words_cn[index,:]
  print("D: ", len(all_words.keys()))
  if subspace == 'without_conceptor':
    #WITHOUT CONCEPTOR
    d_cn = weat_effect_size(career, family, male, female, all_words)
    p_cn = weat_p_value(career, family, male, female, all_words, 1000)
    print("Without conceptor")
    print('WEAT d = ', d_cn)
    print('WEAT p = ', p_cn)
  else:
    #WITH CONCEPTOR
    d_cn = weat_effect_size(career, family, male, female, all_words)
    p_cn = weat_p_value(career, family, male, female, all_words, 1000)
    print("With conceptor: ", embd, subspace)
    print('WEAT d = ', d_cn)
    print('WEAT p = ', p_cn)

  row = [embd, subspace, d_cn, p_cn]
  results.append(row)

  
  
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

# Hard Debiasing

## Mu et. al. Hard Debiasing

In [0]:
from sklearn.decomposition import PCA

#Project off first principal component
def hard_debias(all_words, subspace):
  all_words = np.array(all_words)
  subspace = np.array(subspace)
  print(all_words.shape, " ", subspace.shape)
  pca = PCA(n_components = 1)
  pca.fit(subspace)
  pc1 = np.array(pca.components_)
  
  temp = (pc1.T @ (pc1 @ all_words.T)).T
  ret = all_words - temp
  
  return ret


Test Hard Debiasing using WEAT

In [0]:
import numpy as np
import pandas as pd


resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['glove', 'word2vec', 'fasttext','elmo']
all_subspace = ['without_debiasing', 'gender_list_extended','gender_list_propernouns','gender_list_pronouns', 'gender_list_all']
# all_subspace = ['without_debiasing', 'race_list']

# science = WEATLists.W_8_Science
# arts = WEATLists.W_8_Arts
# male = WEATLists.W_8_Male_terms
# female = WEATLists.W_8_Female_terms

white = WEATLists.W_5_Unused_full_list_European_American_names
black = WEATLists.W_5_Unused_full_list_African_American_names
pleasant = WEATLists.W_5_Pleasant
unpleasant = WEATLists.W_5_Unpleasant

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_debiasing' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_debiasing':
      #Load all embeddings of the subspace as a matrix
      if embd == 'elmo' or embd == 'bert':
        subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
        all_words_cn = hard_debias(all_words_mat, subspace_words_mat)
        print("Subspace mat: ", np.array(subspace_words_mat).shape)
      else:
        subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
        all_words_cn = hard_debias(all_words_mat, subspace_words_mat)
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
    
    if subspace == 'without_debiasing':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("Without debiasing")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("With debiasing: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

## Bolukbasi hard debiasing

In [0]:
def doPCA(pairs, mat, index, num_components = 5):
    matrix = []
    for a, b in pairs:
        center = (mat[index[a.lower()]] + mat[index[b.lower()]])/2
        matrix.append(mat[index[a.lower()]] - center)
        matrix.append(mat[index[b.lower()]] - center)
    matrix = np.array(matrix)
    pca = PCA(n_components = num_components)
    pca.fit(matrix)
    # bar(range(num_components), pca.explained_variance_ratio_)
    return pca

def drop(u, v):
    return u - v * u.dot(v) / v.dot(v)
  
def normalize(all_words_mat):
    all_words_mat /= np.linalg.norm(all_words_mat, axis=1)[:, np.newaxis]
    return all_words_mat

In [0]:
def debias(all_words_mat, all_words_index, gender_specific_words, definitional, equalize):
    gender_direction = doPCA(definitional, all_words_mat, all_words_index).components_[0]
    specific_set = set(gender_specific_words)
    for w in list(all_words_index.keys()):
        if w not in specific_set:
            all_words_mat[all_words_index[w.lower()]] = drop(all_words_mat[all_words_index[w.lower()]], gender_direction)
    all_words_mat = normalize(all_words_mat)
    candidates = {x for e1, e2 in equalize for x in [(e1.lower(), e2.lower()),
                                                     (e1.title(), e2.title()),
                                                     (e1.upper(), e2.upper())]}
    print(candidates)
    for (a, b) in candidates:
        if (a.lower() in all_words_index and b.lower() in all_words_index):
            y = drop((all_words_mat[all_words_index[a.lower()]] + all_words_mat[all_words_index[b.lower()]]) / 2, gender_direction)
            z = np.sqrt(1 - np.linalg.norm(y)**2)
            if (all_words_mat[all_words_index[a.lower()]] - all_words_mat[all_words_index[b.lower()]]).dot(gender_direction) < 0:
                z = -z
            all_words_mat[all_words_index[a.lower()]] = z * gender_direction + y
            all_words_mat[all_words_index[b.lower()]] = -z * gender_direction + y
    all_words_mat = normalize(all_words_mat)
    return all_words_mat


In [0]:
import numpy as np
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

%cd debiaswe_tutorial/debiaswe/
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f) #gender definitional words
print("definitional", defs)
defs_list = []
for pair in defs:
  defs_list.append(pair[0])
  defs_list.append(pair[1])

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f) 
print("Equalize pairs", equalize_pairs)

%cd ../../
!ls

all_embd = ['glove', 'word2vec', 'fasttext']
all_subspace = ['without_debiasing', 'gender_list_extended','gender_list_propernouns','gender_list_pronouns', 'gender_list_all']
#all_subspace = ['without_debiasing', 'race_list']

math = WEATLists.W_7_Math
arts = WEATLists.W_7_Arts
male = WEATLists.W_7_Male_terms
female = WEATLists.W_7_Female_terms

# white = WEATLists.W_5_Unused_full_list_European_American_names
# black = WEATLists.W_5_Unused_full_list_African_American_names
# pleasant = WEATLists.W_5_Pleasant
# unpleasant = WEATLists.W_5_Unpleasant

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_debiasing' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_debiasing':
      #Load all embeddings of the subspace as a matrix
      if embd == 'elmo' or embd == 'bert':
        #subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
        all_words_cn = debias(all_words_mat, all_words_index, subspace_words_list, defs, equalize_pairs)
        print("Subspace mat: ", np.array(subspace_words_mat).shape)
      else:
        #subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
        all_words_cn = debias(all_words_mat, all_words_index, subspace_words_list, defs, equalize_pairs)
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
    
    if subspace == 'without_debiasing':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("Without debiasing")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("With debiasing: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])